### 금융공학 모델링 실습
#### KOSPI200 선물 이론가 산출

In [2]:
import pandas as pd
import numpy as np

In [8]:
# pd.read_excel(엑셀 파일명, 시트명, index_col=칼럼명)
# 시트명 생략가능, 생략하면 1번 시트 / 칼럼명 생략가능, 생략하면 0,1,2..로 인덱싱
# 엑셀파일이 파이썬 노트북 파일과 다른 폴더에 있다면 경로명 붙이기
df = pd.read_excel('C:/Users/starfish/Desktop/파이썬-금융공학/09.2 KOSPI200 F 201712.xlsx', index_col = '일자')
df.head()

,선물가격,지수
일자,,
2017/09/01,308.40,307.78
2017/09/04,305.60,304.19
2017/09/05,304.85,304.06
2017/09/06,304.30,303.18
2017/09/07,308.10,306.97


In [13]:
df.tail()

,선물가격,지수
일자,,
2017-12-08,324.75,324.68
2017-12-11,324.85,325.09
2017-12-12,323.80,324.07
2017-12-13,326.20,326.18
2017-12-14,327.65,324.74


In [9]:
# 인덱스의 데이터 타입을 확인
type(df.index)

pandas.core.indexes.base.Index

In [10]:
# 날짜 형식으로 바꿔줘야함
df.index = pd.to_datetime(df.index)

In [11]:
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex

##### 선물 이론가 산출
  + 앞서 정리한 산출 이론가 공식을 활용 (연속복리이자율)
  + 금융공학에서 1년을 보통 250일 정도로 계산, 여기서는 계산 편의를 위해 252일로 계산
    - 잔존만기: (최종거래일-기준일)/252
    - 이를 파이썬에서 계산하기 위해선 np.busday_count(날짜1,날짜2)를 활용함
    - 결과적으로... __np.busday_count(t0.date(),T.date())/252__

In [12]:
def futures_price(S, r, d ,T, t0):
    t = np.busday_count(t0.date(),T.date()) / 252
    F = S * np.exp((r-d)*t)
    return F

In [15]:
T = pd.to_datetime('2017-12-14') # 최종거래일
r = 0.0225 # 이자율 = 2.25%
d = 0.015 # 배당률 = 1.5%
for n in (df.index):
    t0 = n
    S = df.loc[n,'지수']
    F = futures_price(S, r, d ,T, t0)
    f = '{:,.2f}'.format(F) # 이론가 숫자 포매팅
    # = 정수부분은 천의 자리마다 쉼표 추가, 소수점 아래는 두 자리까지만 표시
    df.loc[n,'이론가1'] = f # '이론가1' 칼럼에 값 추가

##### 보급형 공식을 이용한 선물 이론가 계산
  + 앞서 정리한 산출 이론가 공식을 활용 (단순 이자율)
  + 이와 함께 1년 = 365일로 게산

In [16]:
def S_futures_price(S, r, d ,T, t0):
    t = (T-t0).days / 365 # 잔존만기 계산 (주말까지 포함)
    F = S * (1+(r-d)*t)
    return F

In [17]:
T = pd.to_datetime('2017-12-14')
r = 0.0225
d = 0.015
for n in (df.index):
    t0 = n
    S = df.loc[n,'지수']
    F = S_futures_price(S, r, d ,T, t0)
    f = f = '{:,.2f}'.format(F)
    df.loc[n,'이론가2'] = f

In [18]:
df.head()

,선물가격,지수,이론가1,이론가2
일자,,,,
2017-09-01,308.40,307.78,308.46,308.44
2017-09-04,305.60,304.19,304.85,304.82
2017-09-05,304.85,304.06,304.71,304.68
2017-09-06,304.30,303.18,303.82,303.80
2017-09-07,308.10,306.97,307.61,307.59
